In [1]:
# dependencies
import json
import pandas as pd

# load types file
with open('API_Data/types.json', 'r') as in_file:
    types = json.load(in_file)

## Building the Types Dataframe

I want to build a dataframe where the columns are the multipliers for defense, this way when I want to know what to hit a Pokémon with I just need to multiply the two columns together.

In [2]:
type_list = [types[type]['name'] for type in range(len(types))]

type_df = pd.DataFrame(columns=type_list)

In [3]:

for type_no in range(len(types)):
    type = types[type_no]['name']

    # if it gets super effective damage from another type, add a 2 to that row for the type's column
    if len(types[type_no]['damage_relations']['double_damage_from']) > 0:
        
        for attack_type in range(len(types[type_no]['damage_relations']['double_damage_from'])):
            attack_name = types[type_no]['damage_relations']['double_damage_from'][attack_type]['name']
            type_df.loc[attack_name , type] = 2.0

    # if it gets not very effective damage from another type, add a 0.5 to that row for the type's column
    if len(types[type_no]['damage_relations']['half_damage_from']) > 0:
        
        for attack_type in range(len(types[type_no]['damage_relations']['half_damage_from'])):
            attack_name = types[type_no]['damage_relations']['half_damage_from'][attack_type]['name']
            type_df.loc[attack_name , type] = 0.5

    # if it gets no damage from another type, add a 0 to that row for the type's column
    if len(types[type_no]['damage_relations']['no_damage_from']) > 0:
        
        for attack_type in range(len(types[type_no]['damage_relations']['no_damage_from'])):
            attack_name = types[type_no]['damage_relations']['no_damage_from'][attack_type]['name']
            type_df.loc[attack_name , type] = 0

# reorder indeces so they match the column order, fill 1s in everywhere that isn't super/not very effective
type_df = type_df.reindex(type_list)
type_df.fillna(1, inplace=True)
    

In [4]:
type_df.to_csv('Function_Data/TypeDF.csv')

## Building Attack Function

This will be the core function I end up using in the final app, built here to test easily in a notebook.

In [5]:
# core function for deciding what to attack with
def attack_with(type1, type2=None):

    #monotype    
    if type2 == None:
        damage = type_df[type1]
        damage = damage[damage != 1].sort_values(ascending=False)
        return damage
    #dual type
    else:
        damage = type_df[type1] * type_df[type2]
        damage = damage[damage != 1].sort_values(ascending=False)
        return damage


In [6]:
# test function with my favorite 'mon sableye
attack_with('dark','ghost')

fairy       2.0
poison      0.5
normal      0.0
fighting    0.0
psychic     0.0
dtype: float64